### Notebook to create test data from the prod dataset

In [123]:
import pandas as pd
from langchain_ollama import ChatOllama
from langchain.prompts import ChatPromptTemplate
from langchain.schema import HumanMessage, SystemMessage
pd.set_option('display.max_colwidth', None)

In [124]:
# Create DataFrame from prod CSV

df = pd.read_csv('../data/prod/JEOPARDY_CSV_TOP3_Categories.csv')
df.head(3)

,ShowNumber,AirDate,Round,Category,Value,Question,Answer
0,4931,2/6/2006,Double Jeopardy!,SCIENCE,$400,"At sea level at 70 degrees this travels 1,129 feet per second; it speeds up over 1 foot per sec. for each rising degree",sound
1,4931,2/6/2006,Double Jeopardy!,SCIENCE,$800,"The largest tree, the General Sherman in California, is this type, also called a Sierra Redwood",a sequoia
2,4931,2/6/2006,Double Jeopardy!,SCIENCE,"$1,200","(<a href=""http://www.j-archive.com/media/2006-02-06_DJ_13.jpg"" target=""_blank"">Sarah of the Clue Crew reads from the pole vault at Duke University's track in Durham, NC.</a>) In bending an elastic solid, stress is the force causing deformation & this is the 6-letter term for <a href=""http://www.j-archive.com/media/2006-02-06_DJ_13a.jpg"" target=""_blank"">the deformation</a>",strain


In [125]:
# Drop rows that contain URLs to images
df = df[~df['Question'].str.contains(r'\.jpg|\.jpeg|\.png|\.gif|.wmv', case=False, na=False)]
df.head()

,ShowNumber,AirDate,Round,Category,Value,Question,Answer
0,4931,2/6/2006,Double Jeopardy!,SCIENCE,$400,"At sea level at 70 degrees this travels 1,129 feet per second; it speeds up over 1 foot per sec. for each rising degree",sound
1,4931,2/6/2006,Double Jeopardy!,SCIENCE,$800,"The largest tree, the General Sherman in California, is this type, also called a Sierra Redwood",a sequoia
3,4931,2/6/2006,Double Jeopardy!,SCIENCE,"$1,600",6 elements once known as inert gases are now known by this aristocratic name,noble gases
5,1800,5/29/1992,Double Jeopardy!,SCIENCE,$200,"In metric measurement, 10 millimeters equal 1 of these",a centimeter
6,1800,5/29/1992,Double Jeopardy!,SCIENCE,$400,"Boyle's Law says normally if you double the pressure on a gas, the volume decreases by this amount",one-half


In [126]:
# Initialize the Ollama model
llm_phi3 = ChatOllama(
    model="phi3:3.8b",
        temperature=0
)

In [127]:
# Get unique categories
df['Category'].nunique()

3

In [128]:
df.columns

Index(['ShowNumber', 'AirDate', 'Round', 'Category', 'Value', 'Question',
       'Answer'],
      dtype='object')

In [129]:
df['Category'].value_counts()

Category
LITERATURE          493
SCIENCE             481
AMERICAN HISTORY    413
Name: count, dtype: int64

In [130]:
df.dtypes

ShowNumber     int64
AirDate       object
Round         object
Category      object
Value         object
Question      object
Answer        object
dtype: object

In [131]:
df['Round'].value_counts()

Round
Double Jeopardy!    822
Jeopardy!           532
Final Jeopardy!      33
Name: count, dtype: int64

In [132]:
# Add a new column named MockHumanAnswer
df_updated = df.copy()
df_updated['MockHumanAnswer'] = 'None'
print(len(df_updated))

1387


In [133]:
# Function to create a filtered DataFrame based on criteria
def filter_dataframe(df, Categories, ShowNumber=None, StartDate=None):
    # Initial filtering based on Categories and ShowNumber
    if ShowNumber:
        filtered_df = df[(df['Category'].isin(Categories)) & (df['ShowNumber'] == ShowNumber)]
    else:
        filtered_df = df[df['Category'].isin(Categories)]
    
    # Further filtering based on StartDate
    if StartDate:
        StartDate = pd.to_datetime(StartDate, format='%m/%d/%Y')
        filtered_df = filtered_df[filtered_df['AirDate'] >= StartDate]
    
    # Debug: Print the shape of the DataFrame after filtering
    print(f"Shape after initial filtering: {filtered_df.shape}")
    
    # Group by 'Category' and 'Round' and take 5 rows from each group
    filtered_df = filtered_df.groupby(['Category', 'Round']).head(5)
    
    # Debug: Print the shape of the DataFrame after grouping
    print(f"Shape after grouping: {filtered_df.shape}")
    
    return filtered_df


# Convert AirDate to datetime with the specified format
df_updated['AirDate'] = pd.to_datetime(df_updated['AirDate'], format='%m/%d/%Y')

Categories = ['SCIENCE', 'AMERICAN HISTORY', 'LITERATURE']
filtered_df = filter_dataframe(df_updated, Categories)
filtered_df.head()

Shape after initial filtering: (1387, 8)
Shape after grouping: (45, 8)


,ShowNumber,AirDate,Round,Category,Value,Question,Answer,MockHumanAnswer
0,4931,2006-02-06,Double Jeopardy!,SCIENCE,$400,"At sea level at 70 degrees this travels 1,129 feet per second; it speeds up over 1 foot per sec. for each rising degree",sound,None
1,4931,2006-02-06,Double Jeopardy!,SCIENCE,$800,"The largest tree, the General Sherman in California, is this type, also called a Sierra Redwood",a sequoia,None
3,4931,2006-02-06,Double Jeopardy!,SCIENCE,"$1,600",6 elements once known as inert gases are now known by this aristocratic name,noble gases,None
5,1800,1992-05-29,Double Jeopardy!,SCIENCE,$200,"In metric measurement, 10 millimeters equal 1 of these",a centimeter,None
6,1800,1992-05-29,Double Jeopardy!,SCIENCE,$400,"Boyle's Law says normally if you double the pressure on a gas, the volume decreases by this amount",one-half,None


In [134]:
len(filtered_df)

45

In [135]:
len(df_updated)

1387

In [136]:
# Initialize the Ollama model
llm = ChatOllama(
    # model="phi3:3.8b",
    model = "qwen2.5:3b",
    temperature=1
)

# Define a function to generate answers with typos and homophones
def generate_variations(answer):
    prompt = ChatPromptTemplate.from_messages([
        SystemMessage(content="""You are an expert in creating text variations. 
        Generate one variation for each original answer and repeating one letter in one word. Only return the variation. Do not explain why
        the variation was created.
        """),
        HumanMessage(content=f"Answer: {answer}")
    ])
    response = llm.invoke(prompt.format_messages())
    variations = response.content.split('\n')
    return variations

# Create a list to store the variations
variations_list = []

# Iterate over each row in the allcats_df dataframe
for index, row in filtered_df.iterrows():
    variations = generate_variations(row['Answer'])
    for variation in variations:
        new_row = row.copy()
        new_row['MockHumanAnswer'] = variation
        variations_list.append(new_row)

# Convert the list of variations to a DataFrame
variations_df = pd.DataFrame(variations_list)


In [137]:
# Combine the original dataframe with the variations dataframe
allcats_mocks_df = pd.concat([filtered_df, variations_df], ignore_index=True)

In [138]:
# Display the updated DataFrame with mock human answers
allcats_mocks_df.head(3)

,ShowNumber,AirDate,Round,Category,Value,Question,Answer,MockHumanAnswer
0,4931,2006-02-06,Double Jeopardy!,SCIENCE,$400,"At sea level at 70 degrees this travels 1,129 feet per second; it speeds up over 1 foot per sec. for each rising degree",sound,None
1,4931,2006-02-06,Double Jeopardy!,SCIENCE,$800,"The largest tree, the General Sherman in California, is this type, also called a Sierra Redwood",a sequoia,None
2,4931,2006-02-06,Double Jeopardy!,SCIENCE,"$1,600",6 elements once known as inert gases are now known by this aristocratic name,noble gases,None


In [139]:
specific_question = "The largest tree, the General Sherman in California, is this type, also called a Sierra Redwood"
filtered_rows = allcats_mocks_df[allcats_mocks_df['Question'] == specific_question]
filtered_rows

,ShowNumber,AirDate,Round,Category,Value,Question,Answer,MockHumanAnswer
1,4931,2006-02-06,Double Jeopardy!,SCIENCE,$800,"The largest tree, the General Sherman in California, is this type, also called a Sierra Redwood",a sequoia,None
46,4931,2006-02-06,Double Jeopardy!,SCIENCE,$800,"The largest tree, the General Sherman in California, is this type, also called a Sierra Redwood",a sequoia,a sequeoia


In [140]:
allcats_mocks_df.head()

,ShowNumber,AirDate,Round,Category,Value,Question,Answer,MockHumanAnswer
0,4931,2006-02-06,Double Jeopardy!,SCIENCE,$400,"At sea level at 70 degrees this travels 1,129 feet per second; it speeds up over 1 foot per sec. for each rising degree",sound,None
1,4931,2006-02-06,Double Jeopardy!,SCIENCE,$800,"The largest tree, the General Sherman in California, is this type, also called a Sierra Redwood",a sequoia,None
2,4931,2006-02-06,Double Jeopardy!,SCIENCE,"$1,600",6 elements once known as inert gases are now known by this aristocratic name,noble gases,None
3,1800,1992-05-29,Double Jeopardy!,SCIENCE,$200,"In metric measurement, 10 millimeters equal 1 of these",a centimeter,None
4,1800,1992-05-29,Double Jeopardy!,SCIENCE,$400,"Boyle's Law says normally if you double the pressure on a gas, the volume decreases by this amount",one-half,None


In [141]:
missing_mock_human_answer_count = allcats_mocks_df['MockHumanAnswer'].isna().sum()
print(f"Number of rows with missing MockHumanAnswer: {missing_mock_human_answer_count}")

Number of rows with missing MockHumanAnswer: 0


In [142]:
none_mock_human_answer_count = (allcats_mocks_df['MockHumanAnswer'] == 'None').sum()
print(f"Number of rows with MockHumanAnswer of None: {none_mock_human_answer_count}")

Number of rows with MockHumanAnswer of None: 45


In [143]:
allcats_mocks_df = allcats_mocks_df[allcats_mocks_df['MockHumanAnswer'] != 'None']
allcats_mocks_df.head(3)

,ShowNumber,AirDate,Round,Category,Value,Question,Answer,MockHumanAnswer
45,4931,2006-02-06,Double Jeopardy!,SCIENCE,$400,"At sea level at 70 degrees this travels 1,129 feet per second; it speeds up over 1 foot per sec. for each rising degree",sound,souwnod
46,4931,2006-02-06,Double Jeopardy!,SCIENCE,$800,"The largest tree, the General Sherman in California, is this type, also called a Sierra Redwood",a sequoia,a sequeoia
47,4931,2006-02-06,Double Jeopardy!,SCIENCE,"$1,600",6 elements once known as inert gases are now known by this aristocratic name,noble gases,nobel gases


In [144]:
# Verify counts in DataFrames

# df expected: 
print(f'Length of df: {len(df)}. Expected: 1387.')

# allcats_df expected len: 27. 3^3: questions * 3 categories * 3 rounds
print(f'Length of allcats_df: {len(filtered_df)}. Expected: 45.')

# allcats_mocks_df expected len: len(allcats_df) * 3^2
print(f'Length of allcats_mocks_df: {len(allcats_mocks_df)}. Expected: Same as allcats_df.')

Length of df: 1387. Expected: 1387.
Length of allcats_df: 45. Expected: 45.
Length of allcats_mocks_df: 45. Expected: Same as allcats_df.


In [145]:
# Write out the filtered DataFrame as a CSV for use as testing data
allcats_mocks_df.to_csv('mock_data.csv', index=False)